In [63]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import tensorflow as tf
from konlpy.tag import Mecab
from tqdm import tqdm

## Step 1. 데이터 다운로드

In [64]:
csv_path = os.getenv('HOME') + '/aiffel/Aiffel_Online5/Going-depper06/data/ChatbotData.csv'
csv = pd.read_csv(csv_path)
csv.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [65]:
questions = csv['Q']
answers = csv['A']

In [66]:
for i in range(5):
    print(f'Q:{questions[i]}')
    print(f'A:{answers[i]}')

Q:12시 땡!
A:하루가 또 가네요.
Q:1지망 학교 떨어졌어
A:위로해 드립니다.
Q:3박4일 놀러가고 싶다
A:여행은 언제나 좋죠.
Q:3박4일 정도 놀러가고 싶다
A:여행은 언제나 좋죠.
Q:PPL 심하네
A:눈살이 찌푸려지죠.


## Step 2. 데이터 정제

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 소문자 변환
    sentence = re.sub(r"[^a-zA-Z가-힣0-9?.!,]+", " ", sentence) # 영어, 한글, 숫자, 특수문자 제외하고 제거
    return sentence

## Step 3. 데이터 토큰화

In [6]:
mecab = Mecab()

In [7]:
def build_corpus(src_data, tgt_data): # 소스 문장 데이터와 타켓 문장 데이터 입력으로 받음
    mecab_src_corpus, mecab_tgt_corpus = [], []
    mecab_src_len_list, mecab_tgt_len_list = [], []
    
    for s, t in zip(src_data, tgt_data): # preprocess_sentence()로 정제, 토큰화
        s = mecab.morphs(preprocess_sentence(s))
        t = mecab.morphs(preprocess_sentence(t))
        
        mecab_src_corpus.append(s)
        mecab_tgt_corpus.append(t)
        
        mecab_src_len_list.append(len(s))
        mecab_tgt_len_list.append(len(t))
        
    mecab_num_tokens = mecab_src_len_list + mecab_tgt_len_list
    
    mean_len = np.mean(mecab_num_tokens)
    max_len = np.max(mecab_num_tokens)
    mid_len = np.median([mean_len, max_len]) # 토큰 길이 중간값
    
    # mid_len 이상인 문장 제외
    src_corpus, tgt_corpus = [], []
    for q, a in zip(mecab_src_corpus, mecab_tgt_corpus):
        if len(q) <= mid_len and len(a) <= mid_len:
            if q not in src_corpus and a not in tgt_corpus: # 중복 검사
                src_corpus.append(q)
                tgt_corpus.append(a)
    
    return src_corpus, tgt_corpus

In [8]:
que_corpus, ans_corpus = build_corpus(questions, answers)

In [9]:
print(que_corpus[0], ans_corpus[0])

['12', '시', '땡', '!'] ['하루', '가', '또', '가', '네요', '.']


## Step 4. Agumentation

In [10]:
w2v_path = os.getenv('HOME') + '/aiffel/Aiffel_Online5/Going-depper06/model/ko.bin'
w2v = gensim.models.Word2Vec.load(w2v_path)

In [11]:
def lexical_sub(sentence, word2vec):
    try:
        _from = random.choice(sentence)
        _to = w2v.most_similar(_from)[0][0]
    except:
        return sentence
    
    res = []
    for x in sentence:
        if x is _from: res.append(_to)
        else: res.append(x)

    return res

In [12]:
arg_que_corpus = [lexical_sub(x, w2v) for x in que_corpus]
arg_ans_corpus = [lexical_sub(x, w2v) for x in ans_corpus]

/tmp/ipykernel_341/3786821894.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = w2v.most_similar(_from)[0][0]


In [46]:
for i in range(20):
    print(f"Q : {que_corpus[i]} {arg_que_corpus[i]}")
    print(f"A : {ans_corpus[i]} {arg_ans_corpus[i]}")

Q : ['12', '시', '땡', '!'] ['12', '시', '땡', '!']
A : ['<start>', '하루', '가', '또', '가', '네요', '.', '<end>'] ['하루', '가', '또', '가', '군요', '.']
Q : ['1', '지망', '학교', '떨어졌', '어'] ['1', '지망', '학교의', '떨어졌', '어']
A : ['<start>', '위로', '해', '드립니다', '.', '<end>'] ['위로', '해', '드립니다', '는데']
Q : ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다'] ['3', '박', '4', '일', '살', '러', '가', '고', '싶', '다']
A : ['<start>', '여행', '은', '언제나', '좋', '죠', '.', '<end>'] ['여행', '은', '언제나', '좋', '죠', '는데']
Q : ['ppl', '심하', '네'] ['ppl', '심하', '네']
A : ['<start>', '눈살', '이', '찌푸려', '지', '죠', '.', '<end>'] ['눈살', '이', '찌푸려', '지', '죠', '.']
Q : ['sd', '카드', '망가졌', '어'] ['sd', '단말기', '망가졌', '어']
A : ['<start>', '다시', '새로', '사', '는', '게', '마음', '편해요', '.', '<end>'] ['다시', '새로', '타', '는', '게', '마음', '편해요', '.']
Q : ['sns', '맞', '팔', '왜', '안', '하', '지'] ['sns', '맞', '팔', '과연', '안', '하', '지']
A : ['<start>', '잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.', '<end>'] ['잘', '모르', '고', '있', '을', '수', '때문', '있', '어요', '.']
Q : ['

In [14]:
que_corpus = que_corpus + arg_que_corpus + que_corpus
ans_corpus = ans_corpus + ans_corpus + arg_ans_corpus

## Step 5. 데이터 벡터화

In [15]:
#  타겟 데이터에 <start>, <end> 토큰 추가
ans_corpus = [["<start>"] + ans + ["<end>"] for ans in ans_corpus]

In [16]:
ans_corpus[0]

['<start>', '하루', '가', '또', '가', '네요', '.', '<end>']

In [17]:
# 전체 데이터에 대한 단어 사전 구축, 벡터화
data = que_corpus + ans_corpus

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters=' ', oov_token='<unk>')
tokenizer.fit_on_texts(data)
tensor = tokenizer.texts_to_sequences(data)
tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

In [18]:
VOCAB_SIZE = len(tokenizer.index_word) + 2 # 특수 토큰

In [19]:
tensor.shape

(45822, 25)

In [20]:
enc_train, dec_train = tensor[:22911], tensor[22911:]

## Step 6. 훈련하기

In [21]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [22]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [23]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [24]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [25]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [26]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [27]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [28]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [29]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [30]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [31]:
learning_rate = LearningRateScheduler(512)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [33]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [34]:
n_layers = 6
d_model = 512
n_heads = 8
d_ff = 2048
dropout = 0.3

In [35]:
transformer = Transformer(
    n_layers=n_layers,
    d_model=d_model,
    n_heads=n_heads,
    d_ff=d_ff,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=dropout,
    shared_fc=True,
    shared_emb=True)

EPOCHS = 20
BATCH_SIZE = 64

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = train_step(enc_train[idx:idx+BATCH_SIZE],
                                                                     dec_train[idx:idx+BATCH_SIZE],
                                                                     transformer,
                                                                     optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

Epoch 20: 100%|██████████| 358/358 [01:03<00:00,  5.64it/s, Loss 0.3217]


In [36]:
def evaluate(sentence, model, tokenizer):
        mecab = Mecab()
        sentence = mecab.morphs(preprocess_sentence(sentence))
        sentence = tokenizer.texts_to_sequences(sentence)
        _input = tf.keras.preprocessing.sequence.pad_sequences(
            sentence, maxlen=enc_train.shape[-1], padding='post')

        ids = []
        output = tf.expand_dims([tokenizer.word_index['<start>']], 0)
        
        for i in range(dec_train.shape[-1]):
            enc_padding_mask, combined_mask, dec_padding_mask = generate_masks(_input, output)

            predictions, enc_attns, dec_attns, dec_enc_attns = model(
                _input, output, enc_padding_mask, combined_mask, dec_padding_mask)
            
            predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
            
            if tokenizer.word_index['<end>'] == predicted_id:
                result = ' '.join(tokenizer.sequences_to_texts([ids]))
                return result, enc_attns, dec_attns, dec_enc_attns
            
            ids.append(predicted_id)
            output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)
    
        result = ' '.join(self.tokenizer.sequences_to_texts([ids]))
        return result, enc_attns, dec_attns, dec_enc_attns
        

In [37]:
samples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

In [38]:
def eval_bleu_single(model, src_sentence, tgt_sentence, tokenizer):
    src_tokens = tokenizer.texts_to_sequences(src_sentence)
    tgt_tokens = tokenizer.texts_to_sequences(tgt_sentence)

    if (len(src_tokens) > enc_train.shape[-1]): return None
    if (len(tgt_tokens) > enc_train.shape[-1]): return None

    reference = tgt_sentence.split()
    candidate, _, _, _ = evaluate(src_sentence, model, tokenizer)
    candidate = candidate.split()

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if True:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real: ", reference)
        print("Score: %lf\n" % score)
        
    return score

In [39]:
def eval_bleu(model, tokenizer, src_sentences, tgt_sentence):
        total_score = 0.0
        sample_size = len(src_sentences)

        for idx in tqdm(range(sample_size)):
            score = eval_bleu_single(model, src_sentences[idx], tgt_sentence[idx], tokenizer)
            if not score: continue

            total_score += score

        print("Num of Sample:", sample_size)
        print("Total Score:", total_score / sample_size)

In [40]:
def submit(samples, model, tokenizer):
        print('# 제출')
        print()
        
        print('Translations')
        for s in samples:
            result,_,_,_ = evaluate(s, model, tokenizer)
            print(f'> {i + 1}')
            print(f'Q: {s}')
            print(f'A: {result}')
            print()
            
        print('Hyperparameters')
        print(f'> n_layers: {n_layers}')
        print(f'> d_model: {d_model}')
        print(f'> n_heads: {n_heads}')
        print(f'> d_ff: {d_ff}')
        print(f'> dropout: {dropout}')
        print()
        
        print('Training Parameters')
        print(f'> Batch Size: {BATCH_SIZE}')
        print(f'> Epoch At: {EPOCHS}')

In [41]:
submit(samples, transformer, tokenizer)

# 제출

Translations
> 5
Q: 지루하다, 놀러가고 싶어.
A: 다른 도 곳 을 떠나 보 세요 .

> 5
Q: 오늘 일찍 일어났더니 피곤하다.
A: 오늘 은 힘내 려 하 지 않 아요 .

> 5
Q: 간만에 여자친구랑 데이트 하기로 했어.
A: 휴식 도 필요 해요 .

> 5
Q: 집에 있는다는 소리야.
A: 같이 해 보 세요 .

Hyperparameters
> n_layers: 6
> d_model: 512
> n_heads: 8
> d_ff: 2048
> dropout: 0.3

Training Parameters
> Batch Size: 64
> Epoch At: 20


In [42]:
sample_questions = questions[:10]  
sample_answers = answers[:10] 

In [44]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

eval_bleu(transformer, tokenizer, sample_questions, sample_answers)

 10%|█         | 1/10 [00:00<00:03,  2.28it/s]

Source Sentence:  12시 땡!
Model Prediction:  ['기분', '.']
Real:  ['하루가', '또', '가네요.']
Score: 0.000000



 20%|██        | 2/10 [00:01<00:06,  1.28it/s]

Source Sentence:  1지망 학교 떨어졌어
Model Prediction:  ['아직', '은', '매력', '이', '에요', '.']
Real:  ['위로해', '드립니다.']
Score: 0.000000



 30%|███       | 3/10 [00:02<00:06,  1.10it/s]

...

Source Sentence:  SNS 맞팔 왜 안하지ㅠㅠ
Model Prediction:  ['싫', '지', '않', '으면', '만나', '야죠', '.']
Real:  ['잘', '모르고', '있을', '수도', '있어요.']
Score: 0.000000



 90%|█████████ | 9/10 [00:09<00:01,  1.11s/it]

Source Sentence:  SNS 시간낭비인 거 아는데 매일 하는 중
Model Prediction:  ['싫', '지', '않', '으면', '만나', '야죠', '.']
Real:  ['시간을', '정하고', '해보세요.']
Score: 0.000000



100%|██████████| 10/10 [00:10<00:00,  1.02s/it]

Source Sentence:  SNS 시간낭비인데 자꾸 보게됨
Model Prediction:  ['싫', '지', '않', '으면', '만나', '야죠', '.']
Real:  ['시간을', '정하고', '해보세요.']
Score: 0.000000

Num of Sample: 10
Total Score: 0.017266800427409006


In [57]:
s = '집에 있을래'
result,_,_,_ = evaluate(s, transformer, tokenizer)
print(f'Q:{s} \n A:{result}')

Q:집에 있을래 
 A:같이 해 보 세요 .


In [60]:
s = '집을 나갈 거야'
result,_,_,_ = evaluate(s, transformer, tokenizer)
print(f'Q:{s} \n A:{result}')

Q:집을 나갈 거야 
 A:같이 해 보 세요 .


In [61]:
s = '집은 비어 있어'
result,_,_,_ = evaluate(s, transformer, tokenizer)
print(f'Q:{s} \n A:{result}')

Q:집은 비어 있어 
 A:같이 해 보 세요 .


## 결론

### 루브릭

|평가문항|상세기준|
|:---|:---|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|

### 고찰
> result1  
Q: 지루하다, 놀러가고 싶어.  
A: 다른 도 곳 을 떠나 보 세요 .

> result2  
Q: 오늘 일찍 일어났더니 피곤하다.  
A: 오늘 은 힘내 려 하 지 않 아요 .

> result3  
Q: 간만에 여자친구랑 데이트 하기로 했어.  
A: 휴식 도 필요 해요 .

> result4  
Q: 집에 있는다는 소리야.  
A: 같이 해 보 세요 .

**4번**을 제외하고는 전체적으로 완전히 딴 소리를 하는 것 같지는 않았지만 그렇다고 질문을 완전히 파악한 것 같지도 않았다. **1번**같은 경우에는 조사가 어색한 것을 제외하면 `지루하다, 놀러가고 싶어.` 라는 문장에 `다른 도 곳 을 떠나 보 세요` 라고 적절히 답했고, **4번** 같은 경우에는 질문과 관계없는 답을 했다. 하이퍼 파라미터의 조정이 있었는데도 가장 잘 나온 결과가 기대치에 미치지 못한 이유를 찾아봤을 때, Augmentation 과정에서 `A : ['<start>', '돈', '은', '다시', '들어올', '거', '예요', '.', '<end>'] -> ['돈', '은', '곧바로', '들어올', '거', '예요', '.']`를 예로 들어, 의미가 약간 달라진 문장이 생성되었을 것이라고 추측할 수 있었고 이것이 영향을 미쳤을 수도 있었을 것 같다.  
제대로 답하지 못한 **4번** 대답에 대해 `집`이라는 키워드를 고정하고 임의의 질문을 했을 때 다음과 같이 답했다.
> result4-1  
Q: **집**에 있을래  
A:같이 해 보 세요 .  

>result4-2  
Q: **집**을 나갈 거야  
A:같이 해 보 세요 .  

>result4-3  
Q: **집**은 비어 있어  
A:같이 해 보 세요 .  

세 가지 질문에 모두 동일하게 답한 것으로 보아 `집`이라는 키워드에 같은 대답이 나올 확률이 높음을 알 수 있다. augmentation을 적용하였으나 기본적으로 훈련시킨 데이터의 양이 적고, augmentation은 기존(이번 케이스의 경우 길지 않은) 문장의 단어를 약간 수정하여 데이터의 양을 늘렸기 때문에 해당 키워드에 대한 반응이 고정되었을 수도 있을 것 같다는 생각이 들었다. 이 문제에 대해서는 모델의 매커니즘을 확실히 이해하고 데이터셋이 미치는 영향을 더 조사할 필요가 있겠다.


### 회고
|KPT|내용|
|:---|:---|
|Keep|평가 기준에 맞추어 과제를 수행하고 결과를 도출하였다.|
|Problem|실험 과정을 기록하지 못했고 하이퍼파라미터의 조정 과정도 체계적이지 못했다.|
|Try|다양한 조건 하에서 실험할 때에는 상관관계를 파악하기 쉽도록 적절히 통제하고 기록하도록 한다.|